## **Loading required Libraries**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!wget https://github.com/snikhil17/Road_Safety/raw/main/Data/Tweets%2041k.zip

--2022-01-23 03:01:16--  https://github.com/snikhil17/Road_Safety/raw/main/Data/Tweets%2041k.zip
Resolving github.com (github.com)... 52.69.186.44
Connecting to github.com (github.com)|52.69.186.44|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/snikhil17/Road_Safety/main/Data/Tweets%2041k.zip [following]
--2022-01-23 03:01:16--  https://raw.githubusercontent.com/snikhil17/Road_Safety/main/Data/Tweets%2041k.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6923891 (6.6M) [application/zip]
Saving to: ‘Tweets 41k.zip’

Tweets 41k.zip      100%[===================>]   6.60M  --.-KB/s    in 0.1s    

2022-01-23 03:01:17 (67.7 MB/s) - ‘Tweets 41k.zip’ saved [6923891/6923891]



In [ ]:
!unzip Tweets_41k.zip

Archive:  Tweets_41k.zip
  inflating: Tweets 41k/road_accidents_india_40k_0_200.csv  
  inflating: Tweets 41k/road_accidents_india_40k_1000_to_1200.csv  
  inflating: Tweets 41k/road_accidents_india_40k_10000_to_11000.csv  
  inflating: Tweets 41k/road_accidents_india_40k_11000_to_12000.csv  
  inflating: Tweets 41k/road_accidents_india_40k_1200_to_1600.csv  
  inflating: Tweets 41k/road_accidents_india_40k_12000_to_13000.csv  
  inflating: Tweets 41k/road_accidents_india_40k_13000_to_14000.csv  
  inflating: Tweets 41k/road_accidents_india_40k_14000_to_15000.csv  
  inflating: Tweets 41k/road_accidents_india_40k_15000_to_16000.csv  
  inflating: Tweets 41k/road_accidents_india_40k_1600_to_1800.csv  
  inflating: Tweets 41k/road_accidents_india_40k_16000_to_16400.csv  
  inflating: Tweets 41k/road_accidents_india_40k_16400_to_18000.csv  
  inflating: Tweets 41k/road_accidents_india_40k_1800_to_2000.csv  
  inflating: Tweets 41k/road_accidents_india_40k_18000_to_21000.csv  
  inflating:

In [ ]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import re
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

from nltk.corpus import wordnet
import string
from nltk.stem import WordNetLemmatizer, PorterStemmer


nltk.download('words')

from nltk.corpus import stopwords
nltk.download('stopwords')

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import spacy
nlp = spacy.load('en_core_web_sm')


from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.model_selection import GridSearchCV, StratifiedKFold

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import glob
import os
import pandas as pd
df = pd.concat(map(pd.read_csv, glob.glob(os.path.join('/content/Tweets 41k/', "road_accidents_india_40k_*.csv"))), ignore_index= True).drop('Unnamed: 0', axis = 1)
df.head()

,tweet_content,labels_zero_shot,respective_scores,best_labels_scores,best_labels,sentiments_one_shot_classifier_score,sentiments_one_shot_classifier,cleaned_text,sentiment_via_VADER
0,The thought and action of a road accident free...,"['positive information', 'appreciation', 'nega...","[0.737942636013031, 0.2362614870071411, 0.0117...","{'positive information': 0.737942636013031, 'a...","['positive information', 'appreciation']",{'positive': 0.9731520414352417},positive,thought action road accident free engineering ...,positive
1,"In India, almost 2 lakh people lose their live...","['negative information', 'fatal accident', 'co...","[0.7789489030838013, 0.13117451965808868, 0.03...","{'negative information': 0.7789489030838013, '...","['negative information', 'fatal accident']",{'negative': 0.9603044390678406},negative,almost people lose life road accident build km...,negative
2,@DinakaranNews WRT @DinakaranNews Pls INFO @mk...,"['complaints', 'negative information', 'fatal ...","[0.48437389731407166, 0.2385663390159607, 0.19...","{'complaints': 0.48437389731407166, 'negative ...","['complaints', 'negative information']",{'negative': 0.8926806449890137},negative,info complaint bother yet same road accident s...,negative
3,Good Samaritans who rescue victims of serious ...,"['positive information', 'appreciation', 'nega...","[0.6170728206634521, 0.27085718512535095, 0.05...","{'positive information': 0.6170728206634521, '...","['positive information', 'appreciation']",{'positive': 0.8780285120010376},positive,good rescue victim serious road accident rush ...,positive
4,@rongmeinbong @INCIndia @priyankagandhi Very t...,"['complaints', 'negative information', 'apprec...","[0.6081547141075134, 0.23678579926490784, 0.11...","{'complaints': 0.6081547141075134, 'negative i...","['complaints', 'negative information']",{'negative': 0.8049018979072571},negative,priyankagandhi very true minor road accident p...,negative


## **Cleaning Helper Functions** 
- **Function for decontracted and removing IP adresses and other expressions using Regex**
  - words like don't , won't etc will be converted to do not, will not etc.
  - Emojis, additional lines, email-addresses, website names etc are removed here.
- **Removed words like ``no, not, nor`` from english stopwords**
- **Removing Non-English Words**
  - whatever words nltk corpus has, if given word is in that corpus we will consider the word, else replace with blank.
- **Removing Contextual Stopwords**
  - Using POS-tagging feature of Spacy, removed words which were Pronoun, punctuation, number, adverb  etc.
  - Lemmatizing th words and checking if the given word exist in the list of words needs to be removed (contextual words: obtained after checking the word counts)
- **Remaining Preprocessing of text**
  - Combining all the above functions to use as analyzer in TF-IDF vectorizer. 

In [ ]:
"""Functions to clean text using Regex"""
ip_addr_regex = re.compile(r'\b(?:[0-9]{1,3}\.){3}[0-9]{1,3}\b')
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
def regex_cleaning(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    phrase = re.sub(r"[^a-zA-Z0-9]+", " ", phrase)
    phrase = re.sub(r"\r\n", "", phrase)            # Removing additional line
    phrase = re.sub(r"\n", "", phrase)              # Removing additional line 
    phrase = re.sub(r"\S*@\S*\s?", "", phrase)      # Removing email-addresses 
    phrase = re.sub(r'http\S+', '', phrase)         # Removing website links
    phrase = re.sub(ip_addr_regex, "", phrase)      # Removing IP address link.
    phrase = emoji_pattern.sub(r'', phrase)         # Removing Emojis
    
    return phrase.lower() 

"""Ammending Stopwords list"""
stop_words = stopwords.words('english')
for i in ['nor', 'not', 'no']:
  stop_words.remove(i)

"""Removing Non-English Words"""
nltk_words = set(nltk.corpus.words.words())
def only_eng(row):
  for word in row:
    if word in nltk_words or not word.isalpha():
      word = word
    else:
      word = ""
  return row

"""Helper function for lemmatization_check"""
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

# """Extracting Lemmatized word to take care that words like page, paged, pages can be removed at once."""

def lemmatization_check(word):
  word_pos = nltk.pos_tag(word)
  lemma_word = lemmaObject.lemmatize(word,get_wordnet_pos(word))
  return lemma_word

"""Removing Contextual Stopwords"""
# 'ADJ',
lemmaObject = WordNetLemmatizer()
pos_taggings_to_remove = ['DET', 'ADP','ADV','AUX', 'SCONJ',  'INTJ','PUNCT', 'NUM',  'SPACE', 'X']
some_words_to_remove = ['http', 'year', 'india', 'people', 'every']
def remove_contextual_stopwords(row):
 
  final_row = []
  row = "".join([lemmaObject.lemmatize(word[0],get_wordnet_pos(word[1])) for word in nltk.pos_tag(row)])
  row = nlp(row)
  for word in row:    
    if word.pos_ not in pos_taggings_to_remove and word.text not in some_words_to_remove:
      final_row.append(word.text)
    else:
      continue
  return " ".join(final_row)

"""Remaining Preprocessing of text"""
ps = PorterStemmer()
def final_preprocessing(document):
  document_regex_cleaned = regex_cleaning(document)
  noNonEnglishWords = only_eng(document_regex_cleaned)
  words = [" ".join(nltk.word_tokenize(title)) for title in noNonEnglishWords.split()]                                         # Word Tokenization0
  wordWithoutStopwords = [ps.stem(word) for word in words if word not in stop_words if len(word) > 3]                                   # Remove Stopwords
  vocabulary = " ".join([char for char in wordWithoutStopwords if char not in string.punctuation ])                            # Remove Punctuations
  noContextualStopwords = remove_contextual_stopwords(vocabulary)                                                              # Remove Contextual Stopwords
  return noContextualStopwords.split()                                                                                         # Returning lemmatized-Vocabulary

## **Using Helper functions to clean the data**

In [ ]:
df['cleaned_text'] = df['tweet_content'].apply(regex_cleaning)  # Cleaning using Regex
df['cleaned_text'] = df['cleaned_text'].apply(only_eng)  # Selecting Only English words

## **Contextual Stop words**
- Creating a for loop to store all the comments into a single list:
- In terms of Preprocessing only steps used are remove words less than 3 characters and removed stopwords
- The result is stored in a dictionary, where key is word and value is frequency of the word. 
- This dictionary is convered into DataFrame to observe most frequent words and using that decision is made about selecting **Contextual Stop Words**

In [ ]:
freq_dict ={}
text_list = [text for text in df['cleaned_text']]
for sentence in text_list:
  for word in sentence.split(): 
    
    if word in freq_dict.keys() and len(word) > 3 and word not in stopwords.words('english'):
      freq_dict[word] += 1

    if word not in freq_dict.keys() and len(word) > 3 and word not in stopwords.words('english'):
      freq_dict[word] = 1


count_df = pd.DataFrame( freq_dict.values(), freq_dict.keys()).reset_index().rename({0: "Frequency", "index": "Words"}, axis =1)
count_df.sort_values(by = 'Frequency', ascending = False)[:15]

,Words,Frequency
2,road,45117
5,india,41125
3,accident,22744
27,accidents,20768
13844,http,11147
87,killed,7782
24,people,6408
32,every,4027
272,news,3813
36,deaths,3598


## **Observations:**
---
- We can clearly see that words like **http, year, india, people, every** show up in top 15 words (in terms of frequency).
- We can easily call these words as **Contextual Stop-words** and remove such words from our corpus.
- Using Part of Speech, certain POS (Adjective, Pronoun, Number, Pronoun, Punctuation) words are removed as they are also be considered as **Contextual Stop Words** 

### **Using remove_contextual_stopwords function:**
---
- Remove Contextual words from corpus.


In [ ]:
%%time
df['cleaned_text'] = df['cleaned_text'].apply(remove_contextual_stopwords) # Removing Contextual Stopwords

CPU times: user 14min 17s, sys: 3.43 s, total: 14min 21s
Wall time: 14min 20s


## **Let's check the most-frequent words now:**
- Same method as used above

In [ ]:
freq_dict ={}
text_list = [text for text in df['cleaned_text']]
for sentence in text_list:
  for word in sentence.split(): 
    
    if word in freq_dict.keys() and len(word) > 3 and word not in stopwords.words('english'):
      freq_dict[word] += 1

    if word not in freq_dict.keys() and len(word) > 3 and word not in stopwords.words('english'):
      freq_dict[word] = 1


count_df = pd.DataFrame( freq_dict.values(), freq_dict.keys()).reset_index().rename({0: "Frequency", "index": "Words"}, axis =1)
count_df.sort_values(by = 'Frequency', ascending = False)[:15]

,Words,Frequency
2,road,45117
3,accident,22744
23,accidents,20768
79,killed,7782
257,news,3813
31,deaths,3598
255,injured,3076
877,safety,2832
862,times,2761
909,dies,2338


## **Observation:**
- Above result actually makes sense. as most of our data is non-toxic comment, our data now shows that a few toxic words are present in the top 15 most frequenct words.


In [ ]:
df.head()

,tweet_content,labels_zero_shot,respective_scores,best_labels_scores,best_labels,sentiments_one_shot_classifier_score,sentiments_one_shot_classifier,cleaned_text,sentiment_via_VADER
0,The thought and action of a road accident free...,"['positive information', 'appreciation', 'nega...","[0.737942636013031, 0.2362614870071411, 0.0117...","{'positive information': 0.737942636013031, 'a...","['positive information', 'appreciation']",{'positive': 0.9731520414352417},positive,thought and action road accident free engineer...,positive
1,"In India, almost 2 lakh people lose their live...","['negative information', 'fatal accident', 'co...","[0.7789489030838013, 0.13117451965808868, 0.03...","{'negative information': 0.7789489030838013, '...","['negative information', 'fatal accident']",{'negative': 0.9603044390678406},negative,lakh lose lives road accidents nitin gadkari b...,negative
2,@DinakaranNews WRT @DinakaranNews Pls INFO @mk...,"['complaints', 'negative information', 'fatal ...","[0.48437389731407166, 0.2385663390159607, 0.19...","{'complaints': 0.48437389731407166, 'negative ...","['complaints', 'negative information']",{'negative': 0.8926806449890137},negative,dinakarannews wrt dinakarannews info mkstalin ...,negative
3,Good Samaritans who rescue victims of serious ...,"['positive information', 'appreciation', 'nega...","[0.6170728206634521, 0.27085718512535095, 0.05...","{'positive information': 0.6170728206634521, '...","['positive information', 'appreciation']",{'positive': 0.8780285120010376},positive,good samaritans who rescue victims serious roa...,positive
4,@rongmeinbong @INCIndia @priyankagandhi Very t...,"['complaints', 'negative information', 'apprec...","[0.6081547141075134, 0.23678579926490784, 0.11...","{'complaints': 0.6081547141075134, 'negative i...","['complaints', 'negative information']",{'negative': 0.8049018979072571},negative,rongmeinbong incindia priyankagandhi true mino...,negative


In [ ]:
df.columns

Index(['tweet_content', 'labels_zero_shot', 'respective_scores',
       'best_labels_scores', 'best_labels',
       'sentiments_one_shot_classifier_score',
       'sentiments_one_shot_classifier', 'cleaned_text',
       'sentiment_via_VADER'],
      dtype='object')

In [89]:
df.sentiments_one_shot_classifier.value_counts( normalize=True)

negative    0.840293
positive    0.159707
Name: sentiments_one_shot_classifier, dtype: float64

In [ ]:
features = df['cleaned_text'].values
labels = df['sentiments_one_shot_classifier'].values

## **Let's Try BoW on subset of Data**

In [ ]:
# Create BOW in SKlearn
wordVector = CountVectorizer(analyzer=final_preprocessing)

#Build the Vocabulary
finalWordVectorVocab = wordVector.fit(features[:10])

# # To create BOW
bagOfWords = finalWordVectorVocab.transform(features[:10])

In [ ]:
print(finalWordVectorVocab.get_feature_names_out())

['accid' 'accus' 'action' 'aid' 'aisay' 'amitshah' 'answer' 'anybodi'
 'ashish' 'astronomi' 'averag' 'bajaj' 'bash' 'becom' 'bjp4india' 'bjp4up'
 'block' 'bother' 'bramhraksha' 'broad' 'built' 'case' 'cash' 'caught'
 'caus' 'check' 'cmhfcy9dxm' 'cmotamilnadu' 'colleg' 'complaint' 'control'
 'death' 'deepak' 'detay' 'dinakarannew' 'elig' 'engin' 'famili' 'fatal'
 'file' 'free' 'gadkari' 'gather' 'get' 'give' 'given' 'go' 'golden'
 'good' 'govt' 'hand' 'happen' 'helmet' 'helmetman' 'hijrra' 'hour'
 'human' 'incindia' 'indian' 'info' 'innoc' 'iraianbu' 'journalist'
 'judgement' 'kill' 'lakh' 'lakhimpur' 'lane' 'leader' 'life' 'live'
 'logon' 'lose' 'marwa' 'minist' 'minor' 'mishra' 'mkstalin' 'multi'
 'murder' 'myogiadityanath' 'naiv' 'narendramodi' 'nhrc' 'nitin' 'noida'
 'observ' 'other' 'part' 'person' 'pleasur' 'pothol' 'priyankagandhi'
 'prize' 'problem' 'psycholog' 'rddbumnyuf' 'relat' 'requir' 'rescu'
 'respons' 'reward' 'road' 'roadsafeti' 'rongmeinbong' 'rush' 'samaritan'
 'serio

## **Train-Test Split and instantiating TF-IDF Vectorizer:**
- Used **stratify = labels** in train_test_split so that imbalanced ratio is mentained in train-test-split.
- Used **max_features = 4000** to restrict the vocabulary to 4000.
- Used **min_df = 2** which selects only those words which atleast are seen 2 times.
- Fit and Transformed X_train and Transformed X_test

In [ ]:
%%time
# Splitting the data into Training and Testing set
X_train, X_test, y_train, y_test = train_test_split(features, labels, stratify = labels,   test_size= 0.3,  random_state = 15)

#Create TF-IDF using TfidfVectorizer
tfidfVectorizer = TfidfVectorizer(analyzer=final_preprocessing) # Instantiating Tfidfvectorizer
tfidfObject_train = tfidfVectorizer.fit_transform(X_train)                                        # Fit & transform on Training set
tfidfObject_test = tfidfVectorizer.transform(X_test)                                              # Transform the testing set
print(tfidfObject_train.shape, tfidfObject_test.shape)                                            # Checking Shape of training and testing set

(28480, 37881) (12207, 37881)
CPU times: user 11min 29s, sys: 3.08 s, total: 11min 32s
Wall time: 11min 31s


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

In [60]:
svc = SVC(C=0.001, kernel="linear", class_weight='balanced' )
knc = KNeighborsClassifier()
mnb = MultinomialNB()
dtc = DecisionTreeClassifier()
lrc = LogisticRegression(solver='liblinear', penalty='l1')
rfc = RandomForestClassifier( random_state=7)
abc = AdaBoostClassifier( random_state=7)
bc = BaggingClassifier(random_state=7)
etc = ExtraTreesClassifier(random_state=7)
gbdt = GradientBoostingClassifier(random_state=7)
xgb = XGBClassifier(random_state=7)

In [61]:
clfs = {'SVC':svc,
    'KN' : knc, 
    'NB': mnb, 
    'DT': dtc, 
    'LR': lrc, 
    'RF': rfc, 
    'AdaBoost': abc, 
    'BgC': bc, 
    'ETC': etc,
    'GBDT':gbdt,
    'xgb':xgb
}

In [77]:
def train_classifier(clf,X_train,y_train,X_test,y_test):
    clf.fit(X_train,y_train)
    train_pred = clf.predict(X_train)
    test_pred = clf.predict(X_test)
    accuracy_train = metrics.accuracy_score(y_train,train_pred)
    accuracy_test = metrics.accuracy_score(y_test,test_pred)
    precision_train = metrics.precision_score(y_train,train_pred, labels=['negative', 'positive'], pos_label= 'positive')
    precision_test = metrics.precision_score(y_test,test_pred, labels=['negative', 'positive'], pos_label= 'positive')
    recall_train = metrics.recall_score(y_train,train_pred, labels=['negative', 'positive'], pos_label= 'positive')
    recall_test = metrics.recall_score(y_test,test_pred, labels=['negative', 'positive'], pos_label= 'positive')
    
    return accuracy_train,accuracy_test, precision_train, precision_test, recall_train, recall_test

In [78]:
import warnings
warnings.filterwarnings('ignore')

In [79]:

accuracy_scores_tr = []
accuracy_scores_ts = []
precision_tr_scores = []
precision_ts_scores = []
recall_tr_scores = []
recall_ts_scores = []

for name,clf in clfs.items():
    
    current_accuracy_tr,current_accuracy_ts,precision_tr, precision_ts, recall_tr, recall_ts = train_classifier(clf, tfidfObject_train,
                                                                                                      y_train,tfidfObject_test,y_test)
    
    print("For ",name)
    print("Accuracy Train - ",current_accuracy_tr)
    print("Accuracy Test - ",current_accuracy_ts)
    print("Precision Train - \n",precision_tr)
    print("Precision Test - \n",precision_ts)
    print("Recall Train - \n",recall_tr)
    print("Recall Test - \n",recall_ts)
    
    accuracy_scores_tr.append(current_accuracy_tr)
    accuracy_scores_ts.append(current_accuracy_ts)
    precision_tr_scores.append(precision_tr)
    precision_ts_scores.append(precision_ts)
    recall_tr_scores.append(recall_tr)
    recall_ts_scores.append(recall_ts)
    print()
    print("="*100)
    print()

For  SVC
Accuracy Train -  0.945189606741573
Accuracy Test -  0.890718440239207
Precision Train - 
 0.7525790630813461
Precision Test - 
 0.6202185792349727
Recall Train - 
 0.9784520668425681
Recall Test - 
 0.8148717948717948


For  KN
Accuracy Train -  0.9059691011235955
Accuracy Test -  0.8772835258458261
Precision Train - 
 0.829225352112676
Precision Test - 
 0.7014260249554367
Recall Train - 
 0.5178100263852242
Recall Test - 
 0.4035897435897436


For  NB
Accuracy Train -  0.8828651685393258
Accuracy Test -  0.8709756696977144
Precision Train - 
 0.8914728682170543
Precision Test - 
 0.858508604206501
Recall Train - 
 0.3034300791556728
Recall Test - 
 0.23025641025641025


For  DT
Accuracy Train -  0.9992275280898877
Accuracy Test -  0.8863766691242729
Precision Train - 
 0.998677831643896
Precision Test - 
 0.6557830658550083
Recall Train - 
 0.9964819700967458
Recall Test - 
 0.6076923076923076


For  LR
Accuracy Train -  0.9135884831460674
Accuracy Test -  0.911198492668141

In [85]:
performance_df = pd.DataFrame({'Algorithm':clfs.keys(),'Accuracy Train':accuracy_scores_tr,'Accuracy Test':accuracy_scores_ts,
                               'Precision Train':precision_tr_scores,'Precision Test':precision_ts_scores,
                               'Recall Train':recall_tr_scores,'Recall Test':recall_ts_scores}).style.background_gradient(cmap = 'magma_r', subset = ['Recall Test', 'Precision Test', 'Accuracy Test'	])
performance_df

,Algorithm,Accuracy Train,Accuracy Test,Precision Train,Precision Test,Recall Train,Recall Test
0,SVC,0.945190,0.890718,0.752579,0.620219,0.978452,0.814872
1,KN,0.905969,0.877284,0.829225,0.701426,0.517810,0.403590
2,NB,0.882865,0.870976,0.891473,0.858509,0.303430,0.230256
3,DT,0.999228,0.886377,0.998678,0.655783,0.996482,0.607692
4,LR,0.913588,0.911198,0.824572,0.794158,0.582894,0.599487
5,RF,0.999228,0.912100,0.998239,0.868178,0.996922,0.530256
6,AdaBoost,0.882268,0.881789,0.709576,0.717971,0.444811,0.428205
7,BgC,0.988834,0.904317,0.992777,0.751609,0.936895,0.598974
8,ETC,0.999228,0.913738,0.998678,0.855670,0.996482,0.553333
9,GBDT,0.893364,0.890145,0.843253,0.812308,0.408091,0.406154


In [86]:
%%time
# from sklearn.ensemble import RandomForestClassifier
# model = RandomForestClassifier()
lrc.fit(tfidfObject_train,y_train)

preds_train = etc.predict(tfidfObject_train)                                            # Prediction on training set
preds_test = etc.predict(tfidfObject_test)                                              # Prediction on training set

training_metrics = metrics.classification_report(y_train, preds_train)                    # Getting Classification report of Training set predictions
testing_metrics = metrics.classification_report(y_test, preds_test)                       # Getting Classification report of Test set predictions

print(f"Training Confusion Matrix: \n{metrics.confusion_matrix(y_train, preds_train)}\n") # Printing Confusion Matrix (training)
print(f"Training Classification Report: \n{training_metrics}")                            # Printing Classification report (testing)
print("="*100)
print(f"Testing Confusion Matrix: \n{metrics.confusion_matrix(y_test, preds_test)}\n")    # Printing Confusion Matrix (training)
print(f"Testing Classification Report: \n{testing_metrics}")                              # Printing Classification report (testing)

Training Confusion Matrix: 
[[23926     6]
 [   16  4532]]

Training Classification Report: 
              precision    recall  f1-score   support

    negative       1.00      1.00      1.00     23932
    positive       1.00      1.00      1.00      4548

    accuracy                           1.00     28480
   macro avg       1.00      1.00      1.00     28480
weighted avg       1.00      1.00      1.00     28480

Testing Confusion Matrix: 
[[10075   182]
 [  871  1079]]

Testing Classification Report: 
              precision    recall  f1-score   support

    negative       0.92      0.98      0.95     10257
    positive       0.86      0.55      0.67      1950

    accuracy                           0.91     12207
   macro avg       0.89      0.77      0.81     12207
weighted avg       0.91      0.91      0.91     12207

CPU times: user 10.3 s, sys: 23.9 ms, total: 10.3 s
Wall time: 10.3 s


## **Hyper-parameter tuning**



In [ ]:
"""Params"""
params = {'C': [0.01,0.01,0.05,0.5],                                   # Setting up parameter for SVC to be tried out in GridSearch CV
              # 'degree': [1,4,8]
          }                            
clf = SVC( kernel="linear", class_weight='balanced')                   # initializing SVC
model = GridSearchCV(clf, param_grid = params , cv=StratifiedKFold(5), # GridSearchCV with StratifedKfold and scoring = recall and f1, final decision = recall
                     scoring=['recall', 'f1'], refit='recall')
model.fit(tfidfObject_train,y_train)                                   # Fitting the GridSearchModel on training set

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=SVC(class_weight='balanced', kernel='linear'),
             param_grid={'C': [0.01, 0.01, 0.05, 0.5]}, refit='recall',
             scoring=['recall', 'f1'])

In [ ]:
model.best_estimator_  # Best Estimator after applying GridsearchCV

SVC(C=0.01, class_weight='balanced', kernel='linear')

In [ ]:
model.best_score_ # Best Score after appying GridSearchCV

0.8

### **Prediction using SVC (optimized)**

In [128]:
model = SVC(C=0.01, class_weight='balanced', kernel='linear')
model.fit(tfidfObject_train, y_train)
preds_train = model.predict(tfidfObject_train)                                            # Prediction on training set
preds_test = model.predict(tfidfObject_test)                                              # Prediction on training set

training_metrics = metrics.classification_report(y_train, preds_train)                    # Getting Classification report of Training set predictions
testing_metrics = metrics.classification_report(y_test, preds_test)                       # Getting Classification report of Test set predictions

print(f"Training Confusion Matrix: \n{metrics.confusion_matrix(y_train, preds_train)}\n") # Printing Confusion Matrix (training)
print(f"Training Classification Report: \n{training_metrics}")                            # Printing Classification report (testing)
print("="*100)
print(f"Testing Confusion Matrix: \n{metrics.confusion_matrix(y_test, preds_test)}\n")    # Printing Confusion Matrix (training)
print(f"Testing Classification Report: \n{testing_metrics}")                              # Printing Classification report (testing)

Training Confusion Matrix: 
[[21495  2437]
 [ 1309  3239]]

Training Classification Report: 
              precision    recall  f1-score   support

    negative       0.94      0.90      0.92     23932
    positive       0.57      0.71      0.63      4548

    accuracy                           0.87     28480
   macro avg       0.76      0.81      0.78     28480
weighted avg       0.88      0.87      0.87     28480

Testing Confusion Matrix: 
[[9220 1037]
 [ 551 1399]]

Testing Classification Report: 
              precision    recall  f1-score   support

    negative       0.94      0.90      0.92     10257
    positive       0.57      0.72      0.64      1950

    accuracy                           0.87     12207
   macro avg       0.76      0.81      0.78     12207
weighted avg       0.88      0.87      0.88     12207



## **Saving the model**

In [143]:
suff_  = round(metrics.f1_score(y_test, preds_test,average = 'macro' ), 3)
suff_

0.779

In [145]:
import pickle
"""Save the Bagging CLF model"""
output_file = f'model_svc_{suff_}.bin'

with open(output_file, 'wb') as f_out:
    pickle.dump(model, f_out)